In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
run_name = input()

 pretrained_weight_and_margin


In [3]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import frnn
import wandb
import math
sys.path.append('../..')

from LightningModules.DualEmbedding.Models.vanilla_dual_embedding import VanillaDualEmbedding

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            param.data.fill_(0)
        elif name.startswith("layers.0"):  # The first layer does not have ReLU applied on its input
            param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
        else:
            param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))

## Sweep

In [5]:
with open("dual_embedding_sweep.yaml") as f:
        sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)
with open("dual_embedding_default.yaml") as f:
        default_hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
sweep_configuration = {
    "name": run_name,
    "project": "ITk_barrell_dual_embedding",
    "metric": {"name": "pur", "goal": "maximize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [7]:
def load_dict(model):
    checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
    state_dict = checkpoint["state_dict"]
    names = [i for i in state_dict]
    for i in names:
        state = state_dict[i]
        i = i.replace("input_layer1", "input_layer2")
        i = i.replace("layers1", "layers2")
        i = i.replace("output_layer1", "output_layer2")
        state_dict[i] = state

    model.load_state_dict(state_dict)
    del state_dict
    return model

In [8]:
def training():
    wandb.init()
    model = VanillaDualEmbedding({**default_hparams, **wandb.config})
    # kaiming_init(model)
    model = load_dict(model)
    checkpoint_callback = ModelCheckpoint(
        monitor='pur',
        mode="max",
        save_top_k=2,
        save_last=True)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=default_hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_dual_embedding/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "ITk_barrel_dual_embedding")

# run the sweep
wandb.agent(sweep_id, function=training)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 9ycd43iv
Sweep URL: https://wandb.ai/exatrkx/ITk_barrel_dual_embedding/sweeps/9ycd43iv


wandb: Agent Starting Run: vfydayb3 with config:
wandb: 	margin: 0.5
wandb: 	use_bidir_truth: True
wandb: 	weight_ratio: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'weight_ratio' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'use_bidir_truth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'margin' was locked by 'sweep' (ignored update).

  | Name          | Type             | Params
---------------------------------------------------
0 | cos           | CosineSimilarity | 0     
1 | input_layer1  | Linear           | 12.3 K
2 | layers1       | ModuleList       | 5.2 M 
3 | output_layer1 | Linear           | 16

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'weight_ratio' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'use_bidir_truth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'margin' was locked by 'sweep' (ignored update).

  | Name          | Type             | Params
---------------------------------------------------
0 | cos           | CosineSimilarity | 0     
1 | input_layer1  | Linear           | 12.3 K
2 | layers1       | ModuleList       | 5.2 M 
3 | output_layer1 | Linear           | 16

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/global/homes/r/ryanliu/.conda/envs/gnn/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|█████████▉| 1000/1005 [04:32<00:01,  3.67it/s, loss=0.233, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█████████▉| 1000/1005 [04:33<00:01,  3.66it/s, loss=0.212, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█████████▉| 1000/1005 [04:30<00:01,  3.70it/s, loss=0.21, v_num=aylg] 
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|█████████▉| 1000/1005 [04:28<00:01,  3.72it/s, loss=0.206, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 6: 100%|█████████▉| 1000/1005 [04:25<00:01,  3.76it/s, loss=0.197, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 7: 100%|█████████▉| 1000/1005 [04:27<00:01,  3.74it/s, loss=0.194, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 8: 100%|█████████▉| 1000/1005 [04:26<00:01,  3.75it/s, loss=0.195, v_num=aylg]
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█████████▉| 1000/1005 [04:53<00:01,  3.41it/s, loss=0.215, v_num=xd2u]
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|█████████▉| 1000/1005 [04:39<00:01,  3.58it/s, loss=0.215,

## Construct PyLightning model

In [ ]:
with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
model = VanillaDualEmbedding(hparams)

## Metric Learning

### Train embedding

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
kaiming_init(model)
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
# trainer.fit(model)

## Initialize from trained model

In [ ]:
import wandb
wandb.finish()

with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)
    
hparams["use_dual_encoder"] = True

model = VanillaDualEmbedding(hparams)

checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
state_dict = checkpoint["state_dict"]
names = [i for i in state_dict]
for i in names:
    state = state_dict[i]
    i = i.replace("input_layer1", "input_layer2")
    i = i.replace("layers1", "layers2")
    i = i.replace("output_layer1", "output_layer2")
    state_dict[i] = state

model.load_state_dict(state_dict)
del state_dict

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
# trainer.fit(model)